In [ ]:
# !pip install -U keras
# !pip install tensorflow

In [ ]:
import gensim.downloader
from sklearn.preprocessing import MinMaxScaler

import keras
from keras.models import Sequential
from keras.layers import Dense,Activation

from tensorflow.keras.optimizers import SGD
from tensorflow.keras import layers
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay

import datetime as dt
from scipy.stats import zscore
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

# Primeiro Dataset

In [ ]:
df = pd.read_csv("/content/WeatherEvents_Jan2016-Dec2021.csv")

In [ ]:
df = df[:10000]

In [ ]:
df

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,W-10039,Fog,Severe,2021-02-04 23:35:00,2021-02-04 23:55:00,0.00,US/Central,KAEL,43.6822,-93.3722,Albert Lea,Freeborn,MN,56007.0
9996,W-10040,Snow,Light,2021-02-05 00:35:00,2021-02-05 00:55:00,0.00,US/Central,KAEL,43.6822,-93.3722,Albert Lea,Freeborn,MN,56007.0
9997,W-10041,Fog,Moderate,2021-02-05 15:55:00,2021-02-05 16:35:00,0.00,US/Central,KAEL,43.6822,-93.3722,Albert Lea,Freeborn,MN,56007.0
9998,W-10042,Fog,Severe,2021-02-05 16:35:00,2021-02-05 17:35:00,0.00,US/Central,KAEL,43.6822,-93.3722,Albert Lea,Freeborn,MN,56007.0


## classificação binária: Houve precipitação ou não?

## PreProcessing

In [ ]:
d1 = df
del d1["EventId"]
del d1["AirportCode"]
d1['SeverityType'] = d1['Severity'] + ' ' + d1['Type']
for i in range(0,len(d1['Precipitation(in)'])):
  d1['Precipitation(in)'].loc[i] = 'Precipitation' if d1['Precipitation(in)'].loc[i] > 0.00 and d1['Type'].loc[i] == 'Rain' else 'No Precipitation';
del d1["Type"]
del d1["Severity"]
del d1['State']
del d1['City']
del d1['County']
del d1['ZipCode']

In [ ]:
for i in range(0,len(d1['StartTime(UTC)'])):
  t = d1['StartTime(UTC)'].loc[i].split()
  t2 = d1['EndTime(UTC)'].loc[i].split()
  d1['StartDate'] = t[0]
  d1['StartTime'] = t[1]
  d1['EndDate'] = t2[0]
  d1['EndTime'] = t2[1]

In [ ]:
del d1['StartTime(UTC)']
del d1['EndTime(UTC)']
del d1['TimeZone']
d2 = d1

In [ ]:
df_text = d1[['Precipitation(in)','SeverityType','StartDate','EndDate','StartTime','EndTime']]
d1 = d1.drop(['Precipitation(in)','SeverityType','StartDate','EndDate','StartTime','EndTime'], axis = 1)

Tratamento para dados numéricos


In [ ]:
scaler = MinMaxScaler()
scaler.fit(d1)
X_norm = scaler.transform(d1)
df_norm = pd.DataFrame(X_norm)

## Word2Vec



In [ ]:
word_vectors = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
novo_att = []
for sentence in df_text['SeverityType']:
    lista = []
    lista1 = []
    lista2 = []
    w = sentence.split()
    lista1 = word_vectors[w[0]]
    lista2 = word_vectors[w[1]]
    lista = lista1 + lista2
    novo_att.append(lista)
df_text_num = pd.DataFrame(novo_att)

In [ ]:
novo_df = pd.concat([df_norm,df_text_num], ignore_index=True,axis=1)

In [ ]:
novo_df

,0,1,2,3,4,5,6,7,8,9,...,292,293,294,295,296,297,298,299,300,301
0,0.378312,0.0,0.082031,0.074707,-0.370117,-0.280029,0.255859,0.163574,0.109375,-0.130127,...,0.149902,-0.335938,-0.090820,0.146606,0.189941,-0.150879,-0.222656,0.444336,0.273376,-0.292725
1,0.378312,0.0,0.082031,0.074707,-0.370117,-0.280029,0.255859,0.163574,0.109375,-0.130127,...,0.149902,-0.335938,-0.090820,0.146606,0.189941,-0.150879,-0.222656,0.444336,0.273376,-0.292725
2,0.378312,0.0,0.082031,0.074707,-0.370117,-0.280029,0.255859,0.163574,0.109375,-0.130127,...,0.149902,-0.335938,-0.090820,0.146606,0.189941,-0.150879,-0.222656,0.444336,0.273376,-0.292725
3,0.378312,0.0,0.082031,0.074707,-0.370117,-0.280029,0.255859,0.163574,0.109375,-0.130127,...,0.149902,-0.335938,-0.090820,0.146606,0.189941,-0.150879,-0.222656,0.444336,0.273376,-0.292725
4,0.378312,0.0,0.082031,0.074707,-0.370117,-0.280029,0.255859,0.163574,0.109375,-0.130127,...,0.149902,-0.335938,-0.090820,0.146606,0.189941,-0.150879,-0.222656,0.444336,0.273376,-0.292725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.000000,1.0,-0.124268,0.074585,-0.724609,-0.115967,-0.008789,0.130493,-0.469238,0.331055,...,0.354004,-0.478027,0.180298,0.322266,0.075928,-0.289062,-0.596680,0.244629,0.543945,0.050903
9996,1.000000,1.0,0.082031,0.074707,-0.370117,-0.280029,0.255859,0.163574,0.109375,-0.130127,...,0.149902,-0.335938,-0.090820,0.146606,0.189941,-0.150879,-0.222656,0.444336,0.273376,-0.292725
9997,1.000000,1.0,0.187500,-0.124146,-0.593750,0.115234,-0.092773,0.316040,-0.691406,0.321289,...,0.239746,-0.401855,0.322266,0.313965,-0.007812,-0.400391,-0.189941,0.176758,0.382812,0.487427
9998,1.000000,1.0,-0.124268,0.074585,-0.724609,-0.115967,-0.008789,0.130493,-0.469238,0.331055,...,0.354004,-0.478027,0.180298,0.322266,0.075928,-0.289062,-0.596680,0.244629,0.543945,0.050903


## Modelo

In [ ]:
model = Sequential()
model.add(Dense(256,input_dim=301, activation="relu"))
model.add(Dense(128,activation="relu"))
model.add(Dense(1, activation='linear'))

In [ ]:
y = novo_df[0].values
novo_df = novo_df.drop([0], axis = 1)
X = novo_df.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.5)

In [ ]:
sgd = SGD(learning_rate=0.00002)
model.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=['accuracy']) #modelo de categorização binária
history = model.fit(X_train,y_train,validation_data=(X_valid,y_valid),batch_size=32,epochs=10)

Epoch 1/10
250/250 [==============================] - 1s 3ms/step - loss: 5.2437e-08 - accuracy: 0.4006 - val_loss: 5.2325e-08 - val_accuracy: 0.4170
Epoch 2/10
250/250 [==============================] - 1s 3ms/step - loss: 5.2437e-08 - accuracy: 0.4006 - val_loss: 5.2325e-08 - val_accuracy: 0.4170
Epoch 3/10
250/250 [==============================] - 1s 3ms/step - loss: 5.2437e-08 - accuracy: 0.4006 - val_loss: 5.2325e-08 - val_accuracy: 0.4170
Epoch 4/10
250/250 [==============================] - 1s 3ms/step - loss: 5.2437e-08 - accuracy: 0.4006 - val_loss: 5.2325e-08 - val_accuracy: 0.4170
Epoch 5/10
250/250 [==============================] - 1s 3ms/step - loss: 5.2437e-08 - accuracy: 0.4006 - val_loss: 5.2325e-08 - val_accuracy: 0.4170
Epoch 6/10
250/250 [==============================] - 1s 3ms/step - loss: 5.2437e-08 - accuracy: 0.4006 - val_loss: 5.2325e-08 - val_accuracy: 0.4170
Epoch 7/10
250/250 [==============================] - 1s 4ms/step - loss: 5.2437e-08 - accuracy: 0.4

## Quantidade de precipitação

In [ ]:
d2

,Precipitation(in),LocationLat,LocationLng,SeverityType,StartDate,StartTime,EndDate,EndTime
0,No Precipitation,38.0972,-106.1689,Light Snow,2021-02-05,17:35:00,2021-02-05,18:15:00
1,No Precipitation,38.0972,-106.1689,Light Snow,2021-02-05,17:35:00,2021-02-05,18:15:00
2,No Precipitation,38.0972,-106.1689,Light Snow,2021-02-05,17:35:00,2021-02-05,18:15:00
3,No Precipitation,38.0972,-106.1689,Light Snow,2021-02-05,17:35:00,2021-02-05,18:15:00
4,No Precipitation,38.0972,-106.1689,Light Snow,2021-02-05,17:35:00,2021-02-05,18:15:00
...,...,...,...,...,...,...,...,...
9995,No Precipitation,43.6822,-93.3722,Severe Fog,2021-02-05,17:35:00,2021-02-05,18:15:00
9996,No Precipitation,43.6822,-93.3722,Light Snow,2021-02-05,17:35:00,2021-02-05,18:15:00
9997,No Precipitation,43.6822,-93.3722,Moderate Fog,2021-02-05,17:35:00,2021-02-05,18:15:00
9998,No Precipitation,43.6822,-93.3722,Severe Fog,2021-02-05,17:35:00,2021-02-05,18:15:00


In [ ]:
df_text = d2[['Precipitation(in)','SeverityType','StartDate','EndDate','StartTime','EndTime']]
d2 = d2.drop(['Precipitation(in)','SeverityType','StartDate','EndDate','StartTime','EndTime'], axis = 1)

Tratamento dados numéricos


In [ ]:
scaler = MinMaxScaler()
scaler.fit(d2)
X_norm = scaler.transform(d2)
df_norm = pd.DataFrame(X_norm)

Tratamento para os dados textuais



## One Hot Encoding

In [ ]:
df_dummies = pd.get_dummies(df_text)
novo_df = pd.concat([df_norm,df_dummies], ignore_index=True,axis=1)

In [ ]:
novo_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0.378312,0.0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,1
1,0.378312,0.0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,1
2,0.378312,0.0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,1
3,0.378312,0.0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,1
4,0.378312,0.0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.000000,1.0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1
9996,1.000000,1.0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,1
9997,1.000000,1.0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1
9998,1.000000,1.0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1


In [ ]:
y = novo_df[0].values
novo_df = novo_df.drop([0],axis=1)
X = novo_df.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
model = Sequential()
model.add(Dense(256,input_dim=18, activation="relu"))
model.add(Dense(128,activation="relu"))
model.add(Dense(1, activation='linear'))

In [ ]:
sgd = SGD(learning_rate=0.00002)
model.compile(loss='mean_absolute_error',optimizer=sgd,metrics=['mean_squared_error'])
history_fine = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=30)

Epoch 1/30
210/210 [==============================] - 1s 3ms/step - loss: 0.4123 - mean_squared_error: 0.2861 - val_loss: 0.4065 - val_mean_squared_error: 0.2780
Epoch 2/30
210/210 [==============================] - 1s 3ms/step - loss: 0.4108 - mean_squared_error: 0.2816 - val_loss: 0.4049 - val_mean_squared_error: 0.2737
Epoch 3/30
210/210 [==============================] - 1s 3ms/step - loss: 0.4092 - mean_squared_error: 0.2774 - val_loss: 0.4033 - val_mean_squared_error: 0.2694
Epoch 4/30
210/210 [==============================] - 1s 3ms/step - loss: 0.4076 - mean_squared_error: 0.2731 - val_loss: 0.4017 - val_mean_squared_error: 0.2652
Epoch 5/30
210/210 [==============================] - 1s 3ms/step - loss: 0.4061 - mean_squared_error: 0.2690 - val_loss: 0.4001 - val_mean_squared_error: 0.2612
Epoch 6/30
210/210 [==============================] - 1s 3ms/step - loss: 0.4045 - mean_squared_error: 0.2648 - val_loss: 0.3985 - val_mean_squared_error: 0.2573
Epoch 7/30
210/210 [========

# Segundo Dataset

In [ ]:
df = pd.read_csv("/content/weatherHistory.csv")

In [ ]:
df.dropna()

In [ ]:
df = df[:10000]

In [ ]:
for i in range(0,len(df['Formatted Date'])):
  t = df['Formatted Date'].loc[i].split()
  df['Date'] = t[0]
  df['Time'] = t[1]
del df['Formatted Date']
del df['Daily Summary']

In [ ]:
df

,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Date,Time
0,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,2007-08-28,15:00:00.000
1,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,2007-08-28,15:00:00.000
2,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,2007-08-28,15:00:00.000
3,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,2007-08-28,15:00:00.000
4,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,2007-08-28,15:00:00.000
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Partly Cloudy,rain,22.105556,22.105556,0.67,14.1197,292.0,12.9283,0.0,1015.39,2007-08-28,15:00:00.000
9996,Partly Cloudy,rain,22.844444,22.844444,0.63,15.8585,295.0,13.6528,0.0,1015.13,2007-08-28,15:00:00.000
9997,Partly Cloudy,rain,23.544444,23.544444,0.60,17.4685,298.0,14.3612,0.0,1014.88,2007-08-28,15:00:00.000
9998,Mostly Cloudy,rain,24.183333,24.183333,0.58,18.8692,299.0,14.9730,0.0,1014.65,2007-08-28,15:00:00.000


In [ ]:
df_text = df[['Precip Type', 'Summary', 'Date', 'Time']]
df = df.drop(['Precip Type','Summary','Date', 'Time'], axis = 1)

Tratamento para dados numéricos

In [ ]:
scaler = MinMaxScaler()
scaler.fit(df)
X_norm = scaler.transform(df)
df_norm = pd.DataFrame(X_norm)

## Word2Vec

In [ ]:
novo_att = []
for sentence in df_text['Summary']:
    lista = []
    lista1 = []
    lista2 = []
    w = sentence.split()
    if(len(w) >= 1):
      continue
    if(len(w) == 2):
      lista1 = word_vectors[w[0]]
      lista2 = word_vectors[w[1]]
      lista = lista1 + lista2
    else:
      lista.append(w[0])
    novo_att.append(lista)
df_text_num = pd.DataFrame(novo_att)

In [ ]:
novo_df = pd.concat([df_norm,df_text_num], ignore_index=True,axis=1)

In [ ]:
novo_df

,0,1,2,3,4,5,6,7
0,0.460028,0.487552,0.870588,0.307396,0.699164,0.983,0.0,0.971286
1,0.457750,0.484887,0.835294,0.310550,0.721448,0.983,0.0,0.971765
2,0.458184,0.520441,0.870588,0.085524,0.568245,0.929,0.0,0.972061
3,0.436924,0.463666,0.800000,0.307045,0.749304,0.983,0.0,0.972511
4,0.446035,0.480753,0.800000,0.240449,0.721448,0.983,0.0,0.972607
...,...,...,...,...,...,...,...,...
9995,0.706693,0.730914,0.611765,0.307396,0.813370,0.803,0.0,0.971535
9996,0.721119,0.743133,0.564706,0.345251,0.821727,0.848,0.0,0.971286
9997,0.734787,0.754708,0.529412,0.380301,0.830084,0.892,0.0,0.971047
9998,0.747261,0.765273,0.505882,0.410796,0.832869,0.930,0.0,0.970827


## Modelo

In [ ]:
model = Sequential()
model.add(Dense(256,input_dim=7, activation="relu"))
model.add(Dense(128,activation="relu"))
model.add(Dense(1, activation='softmax'))

In [ ]:
y = novo_df[0].values
novo_df = novo_df.drop([0], axis = 1)
X = novo_df.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, test_size=0.5)

In [ ]:
sgd = SGD(learning_rate=0.00002)
model.compile(loss="mean_squared_error",optimizer=sgd,metrics=['mean_squared_error']) #modelo de categorização binária
history = model.fit(X_train,y_train,validation_data=(X_valid,y_valid),batch_size=32,epochs=10)

Epoch 1/10
250/250 [==============================] - 1s 3ms/step - loss: 0.2770 - mean_squared_error: 0.2770 - val_loss: 0.2834 - val_mean_squared_error: 0.2834
Epoch 2/10
250/250 [==============================] - 1s 2ms/step - loss: 0.2770 - mean_squared_error: 0.2770 - val_loss: 0.2834 - val_mean_squared_error: 0.2834
Epoch 3/10
250/250 [==============================] - 1s 2ms/step - loss: 0.2770 - mean_squared_error: 0.2770 - val_loss: 0.2834 - val_mean_squared_error: 0.2834
Epoch 4/10
250/250 [==============================] - 1s 2ms/step - loss: 0.2770 - mean_squared_error: 0.2770 - val_loss: 0.2834 - val_mean_squared_error: 0.2834
Epoch 5/10
250/250 [==============================] - 1s 2ms/step - loss: 0.2770 - mean_squared_error: 0.2770 - val_loss: 0.2834 - val_mean_squared_error: 0.2834
Epoch 6/10
250/250 [==============================] - 1s 2ms/step - loss: 0.2770 - mean_squared_error: 0.2770 - val_loss: 0.2834 - val_mean_squared_error: 0.2834
Epoch 7/10
250/250 [========

## One-Hot-Encoding

In [ ]:
df = pd.read_csv("/content/weatherHistory.csv")

In [ ]:
df.dropna()
df = df[:10000]

In [ ]:
for i in range(0,len(df['Formatted Date'])):
  t = df['Formatted Date'].loc[i].split()
  df['Date'] = t[0]
  df['Time'] = t[1]
del df['Formatted Date']
del df['Daily Summary']

In [ ]:
df_text = df[['Precip Type', 'Summary', 'Date', 'Time']]
df = df.drop(['Precip Type','Summary','Date', 'Time'], axis = 1)

In [ ]:
scaler = MinMaxScaler()
scaler.fit(df)
X_norm = scaler.transform(df)
df_norm = pd.DataFrame(X_norm)

In [ ]:
df_dummies = pd.get_dummies(df_text)
novo_df = pd.concat([df_norm,df_dummies], ignore_index=True,axis=1)

In [ ]:
novo_df

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0.460028,0.487552,0.870588,0.307396,0.699164,0.983,0.0,0.971286,1,0,...,0,0,0,0,1,0,0,0,1,1
1,0.457750,0.484887,0.835294,0.310550,0.721448,0.983,0.0,0.971765,1,0,...,0,0,0,0,1,0,0,0,1,1
2,0.458184,0.520441,0.870588,0.085524,0.568245,0.929,0.0,0.972061,1,0,...,0,0,1,0,0,0,0,0,1,1
3,0.436924,0.463666,0.800000,0.307045,0.749304,0.983,0.0,0.972511,1,0,...,0,0,0,0,1,0,0,0,1,1
4,0.446035,0.480753,0.800000,0.240449,0.721448,0.983,0.0,0.972607,1,0,...,0,0,1,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.706693,0.730914,0.611765,0.307396,0.813370,0.803,0.0,0.971535,1,0,...,0,0,0,0,1,0,0,0,1,1
9996,0.721119,0.743133,0.564706,0.345251,0.821727,0.848,0.0,0.971286,1,0,...,0,0,0,0,1,0,0,0,1,1
9997,0.734787,0.754708,0.529412,0.380301,0.830084,0.892,0.0,0.971047,1,0,...,0,0,0,0,1,0,0,0,1,1
9998,0.747261,0.765273,0.505882,0.410796,0.832869,0.930,0.0,0.970827,1,0,...,0,0,1,0,0,0,0,0,1,1


In [ ]:
y = novo_df[0].values
novo_df = novo_df.drop([0],axis=1)
X = novo_df.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
model = Sequential()
model.add(Dense(256,input_dim=27, activation="relu"))
model.add(Dense(128,activation="relu"))
model.add(Dense(1, activation='linear'))

In [ ]:
sgd = SGD(learning_rate=0.00002)
model.compile(loss='mean_absolute_error',optimizer=sgd,metrics=['mean_squared_error'])
history_fine = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=30)

Epoch 1/30
210/210 [==============================] - 2s 5ms/step - loss: 0.3812 - mean_squared_error: 0.1857 - val_loss: 0.3567 - val_mean_squared_error: 0.1663
Epoch 2/30
210/210 [==============================] - 1s 3ms/step - loss: 0.3360 - mean_squared_error: 0.1512 - val_loss: 0.3140 - val_mean_squared_error: 0.1350
Epoch 3/30
210/210 [==============================] - 1s 3ms/step - loss: 0.2971 - mean_squared_error: 0.1232 - val_loss: 0.2782 - val_mean_squared_error: 0.1102
Epoch 4/30
210/210 [==============================] - 1s 3ms/step - loss: 0.2643 - mean_squared_error: 0.1008 - val_loss: 0.2480 - val_mean_squared_error: 0.0902
Epoch 5/30
210/210 [==============================] - 1s 3ms/step - loss: 0.2363 - mean_squared_error: 0.0830 - val_loss: 0.2224 - val_mean_squared_error: 0.0745
Epoch 6/30
210/210 [==============================] - 1s 3ms/step - loss: 0.2132 - mean_squared_error: 0.0692 - val_loss: 0.2022 - val_mean_squared_error: 0.0627
Epoch 7/30
210/210 [========